In [44]:
import os
import pdb
import json
import numpy as np
from tqdm import tqdm

# ConceptNet word embeddings -  better than GLOVE!
#WORD_EMBED='/home/brigit/sandbox/sg2im_brigit/word_embeddings/numberbatch-en-19.08.txt'
WORD_EMBED='/home/brigit/sandbox/sg2im_brigit/word_embeddings/glove.6B.300d.txt'
#src = 'conceptnet'
src = 'glove300d'
JSON_VOCAB='/home/brigit/sandbox/sg2im_brigit/datasets/vg/vocab.json'

# load word embeddings
embeddings_dict = {}
head = True
with open(WORD_EMBED, 'r') as f:
  for line in tqdm(f):
    if head:
      head = False
      continue
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], "float32")
    embeddings_dict[word] = vector

400000it [00:21, 18245.34it/s]


In [45]:
# load VG vocab for objects and relationships
with open(JSON_VOCAB) as f:
  vocab = json.load(f)

del vocab['object_name_to_idx']['__image__']
del vocab['pred_name_to_idx']['__in_image__']
objects = list(vocab['object_name_to_idx'].keys())
predicates = list(vocab['pred_name_to_idx'].keys())
predicates_clean = []

#clean predicate list - to deal with cases like 'walking on'
for p in predicates:
   vals = p.split()
   if len(vals) <= 2: # 'walking on', 'eating'
     predicates_clean.append(vals[0])
   elif len(vals) == 3: # 'on top of'
     predicates_clean.append(vals[1])

In [48]:
f = open(src + '_synonyms.txt', "w")

dist_dict = {}
for obj in objects:
  if obj in embeddings_dict:
    dist = []
    for other_obj in objects:
      if other_obj in embeddings_dict:
        d = np.dot(embeddings_dict[obj]/np.linalg.norm(embeddings_dict[obj]), embeddings_dict[other_obj]/np.linalg.norm(embeddings_dict[other_obj]))
        # normalize distance
        d = (d+1)/2
        dist.append(d)
    sort_idx = np.argsort(dist)[::-1]
    sort_dist = np.around(np.array(dist)[sort_idx],3)
    sort_objs = np.array(objects)[sort_idx]
    objs_dist = list(zip(sort_objs[1:10], sort_dist[1:10]))
    print(obj, ':', objs_dist, '\n')
    l = [ "%s (%s)" % x for x in objs_dist ]
    f.write(obj + ': ' + ', '.join(l) + '\n')  
    dist_dict[obj] = objs_dist
  else:
    print('skipping ', obj)

f.close()

out_file = open(src + '_synonyms.json', "w") 
json.dump(dist_dict, out_file, indent = 5 )
out_file.close()

window : [('door', 0.835), ('glass', 0.777), ('roof', 0.77), ('windshield', 0.74), ('ceiling', 0.737), ('floor', 0.733), ('frame', 0.722), ('curtain', 0.708), ('wall', 0.703)] 

tree : [('flower', 0.73), ('leaf', 0.718), ('wood', 0.713), ('trunk', 0.709), ('grass', 0.707), ('plant', 0.677), ('roof', 0.677), ('banana', 0.675), ('fence', 0.666)] 

man : [('woman', 0.85), ('person', 0.822), ('boy', 0.81), ('girl', 0.775), ('hand', 0.741), ('face', 0.726), ('people', 0.722), ('back', 0.708), ('dog', 0.702)] 

shirt : [('jacket', 0.851), ('hat', 0.766), ('coat', 0.731), ('hair', 0.721), ('tie', 0.715), ('logo', 0.714), ('cap', 0.707), ('helmet', 0.707), ('glasses', 0.702)] 

wall : [('street', 0.811), ('fence', 0.722), ('floor', 0.721), ('glass', 0.719), ('ceiling', 0.711), ('roof', 0.711), ('building', 0.708), ('brick', 0.706), ('window', 0.703)] 

person : [('man', 0.822), ('woman', 0.809), ('people', 0.786), ('child', 0.737), ('girl', 0.709), ('boy', 0.707), ('word', 0.702), ('player', 0

boy : [('girl', 0.914), ('man', 0.81), ('child', 0.798), ('woman', 0.773), ('dog', 0.714), ('person', 0.707), ('cat', 0.695), ('hand', 0.672), ('horse', 0.665)] 

helmet : [('jacket', 0.774), ('glove', 0.713), ('hat', 0.712), ('shirt', 0.707), ('glasses', 0.706), ('cap', 0.703), ('motorcycle', 0.698), ('nose', 0.694), ('coat', 0.689)] 

street : [('wall', 0.811), ('road', 0.742), ('sidewalk', 0.729), ('building', 0.716), ('house', 0.692), ('floor', 0.685), ('hill', 0.683), ('part', 0.673), ('line', 0.672)] 

field : [('ball', 0.714), ('ground', 0.706), ('back', 0.678), ('track', 0.676), ('grass', 0.675), ('player', 0.666), ('part', 0.662), ('pole', 0.661), ('spot', 0.659)] 

bottle : [('glasses', 0.765), ('glass', 0.752), ('bag', 0.746), ('water', 0.692), ('box', 0.684), ('towel', 0.678), ('container', 0.673), ('nose', 0.671), ('window', 0.669)] 

glass : [('window', 0.777), ('glasses', 0.756), ('bottle', 0.752), ('roof', 0.746), ('wood', 0.728), ('brick', 0.728), ('floor', 0.725), ('s

coat : [('jacket', 0.792), ('hat', 0.74), ('shirt', 0.731), ('hair', 0.729), ('helmet', 0.689), ('tail', 0.685), ('bag', 0.684), ('towel', 0.683), ('stripe', 0.674)] 

mountain : [('ski', 0.775), ('snow', 0.761), ('hill', 0.754), ('rock', 0.71), ('road', 0.699), ('bear', 0.694), ('bike', 0.682), ('foot', 0.672), ('sand', 0.67)] 

giraffe : [('zebra', 0.799), ('elephant', 0.749), ('paw', 0.649), ('sheep', 0.648), ('pant', 0.623), ('cat', 0.618), ('sock', 0.612), ('tail', 0.612), ('dog', 0.608)] 

sock : [('glove', 0.699), ('shoe', 0.69), ('pillow', 0.643), ('hat', 0.642), ('bag', 0.642), ('shirt', 0.641), ('finger', 0.636), ('paw', 0.636), ('towel', 0.632)] 

cow : [('sheep', 0.798), ('animal', 0.752), ('horse', 0.71), ('bird', 0.71), ('elephant', 0.706), ('dog', 0.702), ('mouth', 0.697), ('cat', 0.669), ('foot', 0.649)] 

counter : [('stand', 0.68), ('food', 0.676), ('hand', 0.676), ('line', 0.657), ('face', 0.652), ('part', 0.649), ('bush', 0.648), ('wave', 0.645), ('clock', 0.643)] 


plane : [('boat', 0.721), ('bus', 0.703), ('train', 0.701), ('engine', 0.693), ('vehicle', 0.692), ('car', 0.691), ('truck', 0.681), ('ground', 0.681), ('back', 0.67)] 

key : [('part', 0.717), ('top', 0.703), ('hand', 0.685), ('back', 0.679), ('bush', 0.679), ('sign', 0.676), ('number', 0.672), ('cabinet', 0.67), ('player', 0.664)] 

tie : [('leg', 0.717), ('shirt', 0.715), ('hat', 0.71), ('cup', 0.69), ('jacket', 0.687), ('hole', 0.676), ('back', 0.672), ('spot', 0.666), ('ball', 0.659)] 



In [47]:
dist_dict



{'window': [('door', 14.007),
  ('glass', 13.38),
  ('roof', 12.365),
  ('ceiling', 11.198),
  ('windshield', 10.996),
  ('floor', 10.555),
  ('frame', 9.387),
  ('curtain', 9.357),
  ('wall', 9.3)],
 'tree': [('flower', 11.629),
  ('leaf', 11.016),
  ('grass', 10.713),
  ('wood', 10.095),
  ('trunk', 10.024),
  ('plant', 9.4),
  ('roof', 8.87),
  ('banana', 8.512),
  ('fence', 8.058)],
 'man': [('woman', 13.582),
  ('boy', 11.993),
  ('person', 11.57),
  ('girl', 11.002),
  ('people', 8.751),
  ('dog', 8.483),
  ('car', 8.351),
  ('face', 8.333),
  ('player', 8.255)],
 'shirt': [('jacket', 17.053),
  ('hat', 12.471),
  ('hair', 11.755),
  ('coat', 11.475),
  ('glasses', 10.778),
  ('logo', 10.724),
  ('tie', 10.146),
  ('flag', 10.012),
  ('cap', 10.001)],
 'wall': [('street', 14.901),
  ('glass', 11.049),
  ('floor', 10.381),
  ('ceiling', 10.335),
  ('fence', 10.196),
  ('brick', 10.093),
  ('roof', 10.058),
  ('stone', 9.33),
  ('window', 9.3)],
 'person': [('woman', 11.819),
  ('m